In [15]:
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Input,Dense,Flatten
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

# In the next cell image augumentation

In [3]:
train_datagen = image.ImageDataGenerator(rotation_range=45,horizontal_flip=True)
val_datagen = image.ImageDataGenerator(rotation_range=45,horizontal_flip=True)

## In next code cell, each directory inside 'train' and 'val' folder is a considered as a class 

In [5]:
training_set=train_datagen.flow_from_directory('./train/',batch_size = 400,target_size=(64, 64))
validation_set=val_datagen.flow_from_directory('./val/images',batch_size = 400,target_size=(64, 64))

Found 90000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [6]:
resnet=ResNet152V2(weights = 'imagenet', input_shape = (64, 64, 3), include_top = False)

234553344/234545216 [==============================] - 89s 0us/step


## Don't train existing layer of transfering learning method, therefore we run the code cell below

In [7]:
for l in resnet.layers:
    l.trainable = False

In [8]:
resnet.summary()

Model: "resnet152v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 34, 34, 64)   0           conv1_conv[0][0]                 
________________________________________________________________________________________

__________________________________________________________________________________________________
conv4_block24_2_bn (BatchNormal (None, 4, 4, 256)    1024        conv4_block24_2_conv[0][0]       
__________________________________________________________________________________________________
conv4_block24_2_relu (Activatio (None, 4, 4, 256)    0           conv4_block24_2_bn[0][0]         
__________________________________________________________________________________________________
conv4_block24_3_conv (Conv2D)   (None, 4, 4, 1024)   263168      conv4_block24_2_relu[0][0]       
__________________________________________________________________________________________________
conv4_block24_out (Add)         (None, 4, 4, 1024)   0           conv4_block23_out[0][0]          
                                                                 conv4_block24_3_conv[0][0]       
__________________________________________________________________________________________________
conv4_bloc

In [20]:
x=Flatten()(resnet.output)

In [21]:
prediction=Dense(200,activation='softmax')(x)

In [22]:
model = Model(inputs = resnet.input, outputs = prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 34, 34, 64)   0           conv1_conv[0][0]                 
______________________________________________________________________________________________

__________________________________________________________________________________________________
conv4_block35_2_relu (Activatio (None, 4, 4, 256)    0           conv4_block35_2_bn[0][0]         
__________________________________________________________________________________________________
conv4_block35_3_conv (Conv2D)   (None, 4, 4, 1024)   263168      conv4_block35_2_relu[0][0]       
__________________________________________________________________________________________________
conv4_block35_out (Add)         (None, 4, 4, 1024)   0           conv4_block34_out[0][0]          
                                                                 conv4_block35_3_conv[0][0]       
__________________________________________________________________________________________________
conv4_block36_preact_bn (BatchN (None, 4, 4, 1024)   4096        conv4_block35_out[0][0]          
__________________________________________________________________________________________________
conv4_bloc

In [23]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

In [ ]:
model.fit_generator(
    training_set, 
    steps_per_epoch = len(training_set),
    epochs = 100,
    validation_data = validation_set,
    validation_steps = len(validation_set),
)